In [7]:
#libraries for analyzing and preparing data 
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 

#libraries for algorithms
from sklearn.pipeline import Pipeline
from sklearn import tree 
from sklearn import datasets 
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier 

#libraries to evaluate performance 
from sklearn.metrics import accuracy_score 
import matplotlib
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from scipy import stats
from scipy.stats import ttest_ind, ttest_rel

In [ ]:
#load data
dfcolumns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
           'hours-per-week', 'native-country', 'income']
df2columns = ['lettr', 'x-box','y-box','width','height','onpix','x-bar','y-bar','x2bar','y2bar','xybar','x2ybar','xy2bar','x-ege','xegvy','y-ege','yegvx']
df3columns = ['Elevation','Aspect','Slope','Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology','Horizontal_Distance_To_Roadways','Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points','Wilderness_Area1','Wilderness_Area2','Wilderness_Area3','Wilderness_Area4',
              'Soil_Type1','Soil_Type2','3','Soil_Type4','Soil_Type5','Soil_Type6','Soil_Type7','Soil_Type8','Soil_Type9','Soil_Type10','Soil_Type11','Soil_Type12','Soil_Type13','Soil_Type14','Soil_Type15','Soil_Type16','Soil_Type17','Soil_Type18','Soil_Type19','Soil_Type20','Soil_Type21','Soil_Type22','Soil_Type23',
              'Soil_Type24','Soil_Type25','Soil_Type26','Soil_Type27','Soil_Type28','Soil_Type29','Soil_Type30','Soil_Type31','Soil_Type32','Soil_Type33','Soil_Type34','Soil_Type35','Soil_Type36','Soil_Type37','Soil_Type38','Soil_Type39','Soil_Type40','Cover_Type']

df = pd.read_csv("adult.data", names=dfcolumns)
df2 = pd.read_csv("letter-recognition.data", names=df2columns)
df3 = pd.read_csv("covtype.data.gz", names=df3columns)

In [ ]:
df.head()
#df2.head()
#df3.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ ?,occupation_ Adm-clerical,...,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,income_ <=50K,income_ >50K
0,39,77516,13,2174,0,40,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,50,83311,13,0,0,13,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,38,215646,9,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,53,234721,7,0,0,40,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,28,338409,13,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
df.shape, df2.shape, df3.shape

((32561, 15), (20000, 17), (581012, 55))

## Data Cleaning
#### Adult dataset

In [ ]:
#encode categorical data 
df = pd.get_dummies(df)
#df

In [ ]:
df.replace("?", np.NaN)
df.dropna()
df.shape

(32561, 110)

## Data Cleaning
#### Letter dataset

In [ ]:
def check(let):
  if (let >= 'A' and let <= 'M'):
    return 1
  else:
    return -1

In [ ]:
df2['lettr'] = df2['lettr'].apply(check)
#df2


In [ ]:
df2 = pd.get_dummies(df2)
#df2

In [ ]:
df2.replace("?", np.NaN)
df2.dropna()
df2.shape

(20000, 17)

## Data Cleaning
#### Cov dataset

In [ ]:
# Finds which value occus the most in the last column
counts = df3['Cover_Type'].value_counts().to_dict() 
print(counts)
#2 is the highest.

{2: 283301, 1: 211840, 3: 35754, 7: 20510, 6: 17367, 5: 9493, 4: 2747}


In [ ]:
def checkThree(num):
  if num == 2:
    return 1
  else:
    return -1

df3['Cover_Type'] = df3['Cover_Type'].apply(checkThree)
df3

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
581008,2391,152,19,67,12,95,240,237,119,845,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
581009,2386,159,17,60,7,90,236,241,130,854,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
581010,2384,170,15,60,5,90,230,245,143,864,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1


# Adult Set

In [ ]:
NUM_TRIALS = 4

#loop for each of the 3 trials
for x in range(NUM_TRIALS):
    #randomly select 5000 samples as training set; reserve rest for testing 
    X_p = df.iloc[:,:-2]
    y_p = df.iloc[:,-2]
    X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                        train_size=5000,
                                                    random_state=12345)
    
    # Initializing Classifiers (knn, logistic regression, decision trees) 
    lr_classifier = LogisticRegression()
    knn_classifier = KNeighborsClassifier()
    dt_classifier = tree.DecisionTreeClassifier()

    # Building the pipelines
    knn_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', knn_classifier)])
        
    lr_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', lr_classifier)])

    dt_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', dt_classifier)])


    #parameters for LR 
    MAX_ITER = 4000 
    reg_params = np.power(10., np.arange(-8,4)) #param for regularization

    #parameters for KNN 
    weights = ['uniform', 'distance']
    k_values = [n*15 for n in range(1,21)] 

    #parameters for DT
    max_depth = np.array([1,2,3,4,5,6,7,8,9,10])


    # Setting up the parameter grids
    lr_param_grid = [{'classifier__penalty': ['11','l2'],
                    'classifier__C': reg_params}]

    knn_param_grid = [{'classifier__weights': weights,
                    'classifier__n_neighbors': k_values}]

    dt_param_grid = [{'classifier__max_depth':max_depth}]


    #scoring metrics
    scoring_metrics = ['accuracy', 'precision', 'recall', 'f1']

    #perform 5-fold cross validation on training set      
    # Setting up multiple GridSearchCV objects, 1 for each algorithm
    gridcvs = {}

    for pgrid, est, name in zip((lr_param_grid, knn_param_grid,dt_param_grid),
                                (lr_pipe, knn_pipe, dt_pipe),
                                ('Logistic_Reg', 'KNN', 'Decision_Tree')):
        gcv = GridSearchCV(estimator=est,
                           param_grid=pgrid,
                           scoring=scoring_metrics,
                           n_jobs=3,
                           cv=5, # just 2-fold inner loop, i.e. train/test
                           verbose=0,
                           return_train_score=True,
                           refit='accuracy')
        gridcvs[name] = gcv
        
    cv_scores = {name: [] for name, gs_est in gridcvs.items()}
    skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    # The outer loop for algorithm selection
    c = 1
    for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
        for name, gs_est in sorted(gridcvs.items()):
            print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

            # The inner loop for hyperparameter tuning
            gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
            y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
            acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
            print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                  (gs_est.best_score_ * 100, acc * 100))
            cv_scores[name].append(acc)

        c += 1

    #find best classifier by looking at results
    for name in cv_scores:
        print('%-8s | outer CV acc. %.2f%% +\- %.3f' % (
              name, 100 * np.mean(cv_scores[name]), 100 * np.std(cv_scores[name])))
    print()
    for name in cv_scores:
        print('{} best parameters'.format(name), gridcvs[name].best_params_)
    
    best_params = gridcvs[name].best_estimator_.fit(X_train, y_train)

outer fold 1/5 | tuning Decision_Tree | inner ACC 84.22% | outer ACC 85.30%
outer fold 1/5 | tuning KNN      | inner ACC 83.67% | outer ACC 83.80%
outer fold 1/5 | tuning Logistic_Reg | inner ACC 84.60% | outer ACC 84.40%
outer fold 2/5 | tuning Decision_Tree | inner ACC 83.95% | outer ACC 84.80%
outer fold 2/5 | tuning KNN      | inner ACC 83.67% | outer ACC 81.90%
outer fold 2/5 | tuning Logistic_Reg | inner ACC 84.95% | outer ACC 84.60%
outer fold 3/5 | tuning Decision_Tree | inner ACC 84.28% | outer ACC 84.50%
outer fold 3/5 | tuning KNN      | inner ACC 82.93% | outer ACC 84.10%
outer fold 3/5 | tuning Logistic_Reg | inner ACC 85.00% | outer ACC 84.20%
outer fold 4/5 | tuning Decision_Tree | inner ACC 84.70% | outer ACC 81.50%
outer fold 4/5 | tuning KNN      | inner ACC 83.52% | outer ACC 82.80%
outer fold 4/5 | tuning Logistic_Reg | inner ACC 84.85% | outer ACC 84.50%
outer fold 5/5 | tuning Decision_Tree | inner ACC 83.98% | outer ACC 84.70%
outer fold 5/5 | tuning KNN      | i

In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['KNN']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['KNN'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

train_scores = {}
test_scores = {}

train_scores['KNN Train Score'] = train_acc
test_scores['KNN Test Score'] = test_acc

Accuracy 83.56% (average over CV test folds)
Best Parameters: {'classifier__n_neighbors': 120, 'classifier__weights': 'distance'}
Training Accuracy: 100.00%
Test Accuracy: 83.15%


In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['Logistic_Reg']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['Logistic_Reg'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

train_scores = {}
test_scores = {}

train_scores['Logistic_Reg Train Score'] = train_acc
test_scores['Logistic_Reg Test Score'] = test_acc

Accuracy 84.80% (average over CV test folds)
Best Parameters: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Training Accuracy: 85.50%
Test Accuracy: 84.72%


In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['Decision_Tree']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['Decision_Tree'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))
# arrays for test scores
train_scores = {}
test_scores = {}

train_scores['Decision_Tree Train Score'] = train_acc
test_scores['Decision_Tree Test Score'] = test_acc

Accuracy 84.30% (average over CV test folds)
Best Parameters: {'classifier__max_depth': 6}
Training Accuracy: 85.46%
Test Accuracy: 83.71%


# Letter Set

In [ ]:
NUM_TRIALS = 3

#loop for each of the 3 trials
for x in range(NUM_TRIALS):
    #randomly select 5000 samples as training set; set aside rest for testing  
    X_p = df2.iloc[:,2:]
    y_p = df2.iloc[:,1]
    X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                        train_size=5000,
                                                    random_state=12345)

    # Initializing Classifiers (logistic regression, knn, random forests & decision trees) 
    lr_classifier = LogisticRegression(max_iter=4000)
    knn_classifier = KNeighborsClassifier()
    dt_classifier = tree.DecisionTreeClassifier()

    # Building the pipelines
    lr_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', lr_classifier)])

    knn_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', knn_classifier)])

    dt_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', dt_classifier)])


    #parameters for LR 
    MAX_ITER = 4000 
    reg_params = np.power(10., np.arange(-8,4)) #param for regularization

    #parameters for KNN 
    weights = ['uniform', 'distance']
    k_values = [n*15 for n in range(1,21)] 

    #parameters for DT
    max_depth = np.array([1,2,3,4,5,6,7,8,9,10])


    # Setting up the parameter grids
    lr_param_grid = [{'classifier__penalty': ['11','l2'],
                    'classifier__C': reg_params}]

    knn_param_grid = [{'classifier__weights': weights,
                    'classifier__n_neighbors': k_values}]

    dt_param_grid = [{'classifier__max_depth':max_depth}]


    #scoring metrics
    scoring_metrics = ['accuracy']

    #perform 5-fold cross validation on training set      
    # Setting up multiple GridSearchCV objects, 1 for each algorithm
    gridcvs = {}

    for pgrid, est, name in zip((lr_param_grid, knn_param_grid,dt_param_grid),
                                (lr_pipe, knn_pipe, dt_pipe),
                                ('Logistic_Reg', 'KNN', 'Decision_Tree')):
        gcv = GridSearchCV(estimator=est,
                           param_grid=pgrid,
                           scoring=scoring_metrics,
                           n_jobs=3,
                           cv=5, # just 2-fold inner loop, i.e. train/test
                           verbose=0,
                           return_train_score=True,
                           refit='accuracy')
        gridcvs[name] = gcv

    cv_scores = {name: [] for name, gs_est in gridcvs.items()}
    skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    # The outer loop for algorithm selection
    c = 1
    for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
        for name, gs_est in sorted(gridcvs.items()):
            print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

            # The inner loop for hyperparameter tuning
            gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
            y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
            acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
            print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                  (gs_est.best_score_ * 100, acc * 100))
            cv_scores[name].append(acc)

        c += 1

    #find best classifier by looking at results
    for name in cv_scores:
        print('%-8s | outer CV acc. %.2f%% +\- %.3f' % (
              name, 100 * np.mean(cv_scores[name]), 100 * np.std(cv_scores[name])))
    print()
    for name in cv_scores:
        print('{} best parameters'.format(name), gridcvs[name].best_params_)
    
    best_params = gridcvs[name].best_estimator_.fit(X_train, y_train)

outer fold 1/5 | tuning Decision_Tree | inner ACC 65.33% | outer ACC 65.30%
outer fold 1/5 | tuning KNN      | inner ACC 52.65% | outer ACC 56.50%
outer fold 1/5 | tuning Logistic_Reg | inner ACC 61.67% | outer ACC 61.00%
outer fold 2/5 | tuning Decision_Tree | inner ACC 65.45% | outer ACC 65.80%
outer fold 2/5 | tuning KNN      | inner ACC 53.90% | outer ACC 55.80%
outer fold 2/5 | tuning Logistic_Reg | inner ACC 61.22% | outer ACC 59.30%
outer fold 3/5 | tuning Decision_Tree | inner ACC 64.85% | outer ACC 63.70%
outer fold 3/5 | tuning KNN      | inner ACC 54.07% | outer ACC 54.10%
outer fold 3/5 | tuning Logistic_Reg | inner ACC 60.82% | outer ACC 62.50%
outer fold 4/5 | tuning Decision_Tree | inner ACC 64.35% | outer ACC 64.30%
outer fold 4/5 | tuning KNN      | inner ACC 53.50% | outer ACC 53.80%
outer fold 4/5 | tuning Logistic_Reg | inner ACC 61.07% | outer ACC 60.30%
outer fold 5/5 | tuning Decision_Tree

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


 | inner ACC 64.40% | outer ACC 64.80%
outer fold 5/5 | tuning KNN     

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


 | inner ACC 52.38% | outer ACC 56.40%
outer fold 5/5 | tuning Logistic_Reg

...

 | inner ACC 64.58% | outer ACC 64.90%
outer fold 5/5 | tuning KNN     

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


 | inner ACC 52.38% | outer ACC 56.40%
outer fold 5/5 | tuning Logistic_Reg

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


 | inner ACC 60.65% | outer ACC 63.10%
Logistic_Reg | outer CV acc. 61.24% +\- 1.397
KNN      | outer CV acc. 55.32% +\- 1.148
Decision_Tree | outer CV acc. 64.70% +\- 0.901

Logistic_Reg best parameters {'classifier__C': 100.0, 'classifier__penalty': 'l2'}
KNN best parameters {'classifier__n_neighbors': 15, 'classifier__weights': 'distance'}
Decision_Tree best parameters {'classifier__max_depth': 9}


In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['KNN']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['KNN'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

train_scores = {}
test_scores = {}

train_scores['KNN Train Score'] = train_acc
test_scores['KNN Test Score'] = test_acc

Accuracy 55.18% (average over CV test folds)
Best Parameters: {'classifier__n_neighbors': 15, 'classifier__weights': 'distance'}
Training Accuracy: 99.58%
Test Accuracy: 56.29%


In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['Logistic_Reg']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['Logistic_Reg'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

train_scores = {}
test_scores = {}

train_scores['Logistic_Reg Train Score'] = train_acc
test_scores['Logistic_Reg Test Score'] = test_acc

Accuracy 61.36% (average over CV test folds)
Best Parameters: {'classifier__C': 10.0, 'classifier__penalty': 'l2'}
Training Accuracy: 63.36%
Test Accuracy: 61.00%


In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['Decision_Tree']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['Decision_Tree'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))
# arrays for test scores
train_scores = {}
test_scores = {}

train_scores['Decision_Tree Train Score'] = train_acc
test_scores['Decision_Tree Test Score'] = test_acc

Accuracy 64.84% (average over CV test folds)
Best Parameters: {'classifier__max_depth': 8}
Training Accuracy: 73.28%
Test Accuracy: 65.09%


# Cov Set

In [ ]:
NUM_TRIALS = 3

#loop for each of the 3 trials
for x in range(NUM_TRIALS):
    #randomly select 5000 samples as training set; set aside rest for testing  
    X_p = df3.iloc[:,:-1]
    y_p = df3.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                        train_size=5000,
                                                    random_state=12345)
    
    # Initializing Classifiers (logistic regression, knn, random forests & decision trees) 
    lr_classifier = LogisticRegression()
    knn_classifier = KNeighborsClassifier()
    dt_classifier = tree.DecisionTreeClassifier()

    # Building the pipelines
    lr_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', lr_classifier)])

    knn_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', knn_classifier)])

    dt_pipe = Pipeline([('std', StandardScaler()),
                      ('classifier', dt_classifier)])


    #parameters for LR 
    MAX_ITER = 4000 
    reg_params = np.power(10., np.arange(-8,4)) #param for regularization

    #parameters for KNN 
    weights = ['uniform', 'distance']
    k_values = [n*15 for n in range(1,21)] 

    #parameters for DT
    max_depth = np.array([1,2,3,4,5,6,7,8,9,10])


    # Setting up the parameter grids
    lr_param_grid = [{'classifier__penalty': ['11','l2'],
                    'classifier__C': reg_params}]

    knn_param_grid = [{'classifier__weights': weights,
                    'classifier__n_neighbors': k_values}]

    dt_param_grid = [{'classifier__max_depth':max_depth}]


    #scoring metrics
    scoring_metrics = ['accuracy', 'precision', 'recall', 'f1']

    #perform 5-fold cross validation on training set      
    # Setting up multiple GridSearchCV objects, 1 for each algorithm
    gridcvs = {}

    for pgrid, est, name in zip((lr_param_grid, knn_param_grid,dt_param_grid),
                                (lr_pipe, knn_pipe, dt_pipe),
                                ('Logistic_Reg', 'KNN', 'Decision_Tree')):
        gcv = GridSearchCV(estimator=est,
                           param_grid=pgrid,
                           scoring=scoring_metrics,
                           n_jobs=3,
                           cv=5, # just 2-fold inner loop, i.e. train/test
                           verbose=0,
                           return_train_score=True,
                           refit='accuracy')
        gridcvs[name] = gcv

    cv_scores = {name: [] for name, gs_est in gridcvs.items()}
    skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    # The outer loop for algorithm selection
    c = 1
    for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
        for name, gs_est in sorted(gridcvs.items()):
            print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

            # The inner loop for hyperparameter tuning
            gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
            y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
            acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
            print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                  (gs_est.best_score_ * 100, acc * 100))
            cv_scores[name].append(acc)

        c += 1
    #find best classifier by looking at results
    for name in cv_scores:
        print('%-8s | outer CV acc. %.2f%% +\- %.3f' % (
              name, 100 * np.mean(cv_scores[name]), 100 * np.std(cv_scores[name])))
    print()
    for name in cv_scores:
        print('{} best parameters'.format(name), gridcvs[name].best_params_)
    
    best_params = gridcvs[name].best_estimator_.fit(X_train, y_train)

outer fold 1/5 | tuning Decision_Tree | inner ACC 76.05% | outer ACC 73.70%
outer fold 1/5 | tuning KNN      | inner ACC 76.02% | outer ACC 77.40%
outer fold 1/5 | tuning Logistic_Reg | inner ACC 74.98% | outer ACC 76.90%
outer fold 2/5 | tuning Decision_Tree | inner ACC 75.38% | outer ACC 74.80%
outer fold 2/5 | tuning KNN      | inner ACC 75.50% | outer ACC 75.90%
outer fold 2/5 | tuning Logistic_Reg | inner ACC 76.27% | outer ACC 74.20%
outer fold 3/5 | tuning Decision_Tree | inner ACC 74.55% | outer ACC 78.00%
outer fold 3/5 | tuning KNN      | inner ACC 74.65% | outer ACC 76.80%
outer fold 3/5 | tuning Logistic_Reg | inner ACC 74.98% | outer ACC 76.30%
outer fold 4/5 | tuning Decision_Tree | inner ACC 75.30% | outer ACC 75.80%
outer fold 4/5 | tuning KNN      | inner ACC 75.77% | outer ACC 75.20%
outer fold 4/5 | tuning Logistic_Reg | inner ACC 75.65% | outer ACC 75.80%
outer fold 5/5 | tuning Decision_Tree | inner ACC 74.45% | outer ACC 74.00%
outer fold 5/5 | tuning KNN      | i

In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['KNN']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['KNN'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

train_scores = {}
test_scores = {}

train_scores['KNN Train Score'] = train_acc
test_scores['KNN Test Score'] = test_acc

Accuracy 76.48% (average over CV test folds)
Best Parameters: {'classifier__n_neighbors': 15, 'classifier__weights': 'distance'}
Training Accuracy: 100.00%
Test Accuracy: 77.48%


In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['Logistic_Reg']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['Logistic_Reg'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

train_scores = {}
test_scores = {}

train_scores['Logistic_Reg Train Score'] = train_acc
test_scores['Logistic_Reg Test Score'] = test_acc

Accuracy 75.52% (average over CV test folds)
Best Parameters: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Training Accuracy: 76.48%
Test Accuracy: 74.93%


In [ ]:
#with best classifier, train on all training data 
# Fitting a model to the whole training using the "best" algorithm
best_algo = gridcvs['Decision_Tree']
best_algo.fit(X_train, y_train)

train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))


#get test set error of this classifier    
print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['Decision_Tree'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))
# arrays for test scores
train_scores = {}
test_scores = {}

train_scores['Decision_Tree Train Score'] = train_acc
test_scores['Decision_Tree Test Score'] = test_acc

Accuracy 76.20% (average over CV test folds)
Best Parameters: {'classifier__max_depth': 10}
Training Accuracy: 85.74%
Test Accuracy: 77.30%


# Ttest

In [8]:
knn = [0.8306,0.8306,0.8306,0.5532,0.5532,0.5532,0.7608,0.7608,0.7608]
lr  = [0.845,0.845,0.845,0.6124,0.6124,0.6124,0.7574,0.7574,0.7574]
dt  = [0.8416,0.842,0.842,0.6478,0.647,0.647,0.7526,0.7514,0.7556]

In [9]:
stats.ttest_ind(lr,dt)

Ttest_indResult(statistic=-0.20830791887880992, pvalue=0.8376171407451202)

In [10]:
stats.ttest_ind(lr,knn)

Ttest_indResult(statistic=0.435653065454439, pvalue=0.6689087390797712)

In [11]:
stats.ttest_ind(dt,knn)

Ttest_indResult(statistic=0.6481735983845635, pvalue=0.5260660604880167)

In [12]:
t,p = ttest_rel(lr,dt)

In [13]:
t

-1.426282783789128

In [14]:
p

0.19162201016244912

In [15]:
knn_acc = [.8356,.5518,.7648]
lr_acc = [.848,.6136,.7552]
dt_acc = [.843, .6484, .762]


In [16]:
stats.ttest_ind(lr_acc,dt_acc)

Ttest_indResult(statistic=-0.13787248198004162, pvalue=0.8970031045060598)

In [17]:
stats.ttest_ind(lr_acc,knn_acc)

Ttest_indResult(statistic=0.1972439365054618, pvalue=0.8532539613184058)

In [18]:
stats.ttest_ind(dt_acc,knn_acc)

Ttest_indResult(statistic=0.3298506968489816, pvalue=0.7580638975682193)

In [19]:
t2,p2 = ttest_rel(lr_acc,dt_acc)

In [20]:
t2

-1.0337003540384972

In [21]:
p2

0.40989536781032143